#Recomendación

## Importación libreria

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import os

import warnings
warnings.filterwarnings("ignore")

# time calculation to track some processes
import time

import pickle

# import the function to compute cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

import scipy as sp

## Importación datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ruta= '/content/drive/MyDrive/Colab Notebooks/Nuclio - Data Science/TFM/df_grande_clusters.pkl'
ruta2= '/content/drive/MyDrive/Colab Notebooks/Nuclio - Data Science/TFM/Pickle_clusters.pkl'
PATH_DATA = '/content/drive/MyDrive/Colab Notebooks/Nuclio - Data Science/TFM/'

Pickle del clustering - con todas las columnas

In [4]:
import pickle

with open(ruta, 'rb') as handle:
    df_info = pickle.load(handle)

Pickle del clustering - solo con las columnas del cluster

In [5]:
import pickle

with open(ruta2, 'rb') as handle:
    df_group = pickle.load(handle)

Datos de los productos

In [6]:
products = pd.read_csv(PATH_DATA+'products_df.csv', encoding='utf-8')
products.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
df_group.head()

,age,meses_recency,mean_M_entre_altas,total_revenue,prods_todo_1s,active_mean,n_prod_probados,total_altas,cuenta_uso_medio,ahorro_uso_medio,financiacion_uso_medio,cluster
pk_cid,,,,,,,,,,,,
15891,59,-1,-1,0,0,0,1,0,0,0,0,2
16063,62,-1,-1,0,0,0,0,0,0,0,0,2
16203,70,4,1,10,0,0,1,1,0,0,0,2
16502,58,4,2,20,0,0,2,2,0,0,0,2
17457,54,0,3,340,1,1,3,6,1,0,0,5


In [8]:
df_info.head()

,pk_cid,active_mean,entry_date,payroll,age,short_term_deposit_altas,short_term_deposit_bajas,short_term_deposit_revenue,loans_altas,loans_bajas,loans_revenue,mortgage_altas,mortgage_bajas,mortgage_revenue,funds_altas,funds_bajas,funds_revenue,securities_altas,securities_bajas,securities_revenue,long_term_deposit_altas,long_term_deposit_bajas,long_term_deposit_revenue,credit_card_altas,credit_card_bajas,credit_card_revenue,payroll_account_altas,payroll_account_bajas,payroll_account_revenue,emc_account_altas,emc_account_bajas,emc_account_revenue,debit_card_altas,debit_card_bajas,debit_card_revenue,em_acount_altas,em_acount_bajas,em_acount_revenue,pension_plan_altas,pension_plan_bajas,pension_plan_revenue,cuenta_altas,cuenta_bajas,cuenta_revenue,ahorro_altas,ahorro_bajas,ahorro_revenue,financiacion_altas,financiacion_bajas,financiacion_revenue,n_prods_con_altas,max_recurrencia,total_altas,total_bajas,total_revenue,min_M_entre_altas,max_M_entre_altas,mean_M_entre_altas,meses_recency,meses_usuario,cuenta_prod_probados,cuenta_todo_1s,ahorro_prod_probados,ahorro_todo_1s,financiacion_prod_probados,financiacion_todo_1s,n_prod_probados,prods_todo_1s,cuenta_uso_medio,ahorro_uso_medio,financiacion_uso_medio
0,15891,0.500000,2015-09,0.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,2,1,0,0,0,0,0,1,0,0.500000,0.000000,0.000000
1,16063,0.714286,2015-09,0.0,62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,7,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
2,16203,0.833333,2015-09,0.0,70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1.0,0.0,10.0,1.0,1.0,1.0,4.0,6,1,0,0,0,0,0,1,0,0.833333,0.000000,0.000000
3,16502,0.888889,2015-09,0.0,58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,2.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,2.0,0.0,20.0,1.0,3.0,2.0,4.0,9,2,0,0,0,0,0,2,0,0.722222,0.000000,0.000000
4,17457,1.000000,2015-09,0.0,54,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,40.0,5.0,4.0,300.0,2,0,6.0,5.0,340.0,2.0,8.0,3.2,0.0,17,1,1,1,0,1,0,3,1,1.000000,0.941176,0.294118


In [9]:
products.head()

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1


## Construcción dataset recomendación

Unimos productos con clusters

In [10]:
df_group = df_group.reset_index()

In [11]:
full_df = pd.merge(products,df_group[['pk_cid','cluster']],on='pk_cid',how='left')

Miramos solo los clusters 4 y 5, ya que son los que más gastan y estos serán nuestros targets

In [12]:
full_df = full_df[(full_df['cluster']==4) | (full_df['cluster']==5)]

Calculamos los productos que tiene cada cliente

In [13]:
full_df["total"] = full_df.iloc[:,2:-1].sum(axis=1)

Calculamos el total de todas las altas que han habido

In [14]:
test = full_df.groupby('pk_cid')['total'].sum()

In [15]:
full_df = pd.merge(full_df,test,on = 'pk_cid')

In [16]:
full_df = full_df.rename(columns={'total_x': 'total_mes', 'total_y': 'total_sum'})

Eliminamos la columna payroll ya que no aporta beneficios 


In [17]:
full_df = full_df[['pk_cid','pk_partition','short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit',
       'credit_card', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount','cluster','total_mes','total_sum']]

Agrupamos por la suma de todos los productos que ha contratado cada cliente en EasyMoney y ordenamos por orden descendente para quedarnos con los clientes que más productos han contratado con EasyMoney

In [18]:
cliente_total = full_df.groupby('pk_cid')[['total_sum']].max()

In [19]:
cliente_total = cliente_total.sort_values('total_sum',ascending = False)

In [20]:
cliente_total

,total_sum
pk_cid,
1128353,149.0
1190607,145.0
1070525,142.0
1233988,136.0
1054409,136.0
...,...
1255271,0.0
1020270,0.0
1020271,0.0


Tenemos que enviar 10.000 emails -> nos quedamos con los 10.000 clientes que más han contratado

In [21]:
cliente_mail = cliente_total[:10100] #filtramos por 100 clientes más porque hay clientes que no estan en el ultimo pk_partition

Filtramos la lista de clientes en el Dataframe principal

In [22]:
cliente_mail

,total_sum
pk_cid,
1128353,149.0
1190607,145.0
1070525,142.0
1233988,136.0
1054409,136.0
...,...
1059321,49.0
1224656,49.0
1251684,49.0


In [23]:
target_clients = full_df[full_df['pk_cid'].isin(cliente_mail.index)]

In [24]:
target_clients['pk_cid'].nunique()

10100

Miramos que productos ha contratado cada cliente durante el tiempo que ha estado con nosotros

In [25]:
target_df = target_clients.groupby('pk_cid')[['short_term_deposit', 'loans', 'mortgage',
       'funds', 'securities', 'long_term_deposit',
       'credit_card', 'pension_plan', 'payroll_account',
       'emc_account', 'debit_card', 'em_account_p', 'em_acount']].sum()

2 approaches: 
  

*   Mirar que productos suele contratar el cliente y mirar si actualmente tiene contratado esos productos -> si no lo tiene, recomendarle el producto que más haya contratado con nosotros
*   Mirar que productos contratan esos clientes y hacer un ranking de los productos e ir recomendando los productos más demandado que no tenga contratado un cliente.



Nos decantamos por el 2n approach

Reducimos el tamaño

In [26]:
target_df = target_df.astype('int8')

Cosine similarity - obtenemos matriz de cosinus para product based y client based similarity

In [27]:
FILE_NO_EXISTS = False
if FILE_NO_EXISTS:

    st = time.time()
    target_df_norm = target_df.apply(lambda x: (x - np.mean(x))/(np.max(x) - np.min(x)), axis = 1)
    et = time.time()
    print("Normalization took {} minutes".format(round((et - st)/60, 2)))

    sparse_ratings = sp.sparse.csr_matrix(target_df_norm.values)

    st = time.time()
    
    product_similarity = cosine_similarity(sparse_ratings.T)
    product_sim_df = pd.DataFrame(product_similarity, index = target_df_norm.columns, columns = target_df_norm.columns)
    product_sim_df.to_pickle(os.path.join(PATH_DATA, "product_similarity.pkl"))

    user_similarity = cosine_similarity(sparse_ratings)
    user_sim_df = pd.DataFrame(user_similarity, index = target_df_norm.index, columns = target_df_norm.index)
    user_sim_df.to_pickle(os.path.join(PATH_DATA, "user_similarity.pkl"))
    
    et = time.time()
    print("Total time to calculate similarity took {} minutes.".format(round((et - st)/60, 2)))
    
else:
    user_sim_df = pd.read_pickle(os.path.join(PATH_DATA, "user_similarity.pkl"))
    product_sim_df = pd.read_pickle(os.path.join(PATH_DATA, "product_similarity.pkl"))

In [28]:
user_sim_df.head()

pk_cid,17970,26018,31624,32376,45370,45850,46327,51389,53420,55339,56347,62064,62088,62175,62943,66463,69294,69575,74214,74455,79129,88904,91972,94134,97322,98344,98588,99023,102614,105831,106220,106944,113436,115885,120751,124619,125189,128298,138351,145461,...,1398684,1398799,1398830,1399255,1399438,1399509,1399519,1399618,1399678,1399727,1399792,1399962,1400200,1400444,1400459,1400493,1400827,1401095,1401406,1401478,1401768,1401841,1402199,1402679,1402957,1402972,1403295,1403420,1404593,1404862,1405769,1405967,1406189,1408595,1410579,1428596,1433369,1437442,1442421,1479480
pk_cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17970,1.000000,-0.404410,0.813570,0.809146,0.603646,0.301374,0.979308,0.666489,0.810498,0.244917,0.590522,0.801772,0.986219,0.810515,0.788464,0.469330,0.459527,0.347740,0.984098,0.812619,0.812052,0.823946,0.140762,0.665331,0.295237,-0.067325,0.768261,0.816434,0.661455,0.929879,0.813570,0.813570,0.611712,0.244274,0.624175,0.581106,0.973483,0.795028,0.904534,0.726288,...,0.676924,0.905561,0.794508,0.824236,0.490168,0.803464,0.981072,0.690242,0.977435,0.649595,0.723233,0.654828,0.981356,0.273705,0.831851,0.605927,0.803610,0.978582,0.950438,0.987817,0.980227,0.947648,0.373421,0.843999,0.749368,0.891337,0.738822,0.968109,0.516694,0.398657,0.157634,0.749497,0.757899,0.446604,0.400236,0.931979,0.813755,0.357907,0.491531,0.652432
26018,-0.404410,1.000000,-0.321171,-0.299820,0.027397,0.098750,-0.421315,-0.389475,-0.329352,0.420969,-0.136821,-0.316513,-0.389327,-0.319964,-0.311259,-0.079617,-0.319718,0.111700,-0.388489,-0.320795,-0.462917,-0.288392,0.470934,0.026156,-0.215018,0.742585,-0.459793,-0.462938,-0.002660,-0.447879,-0.321171,-0.321171,-0.383798,0.077747,-0.349071,-0.006332,-0.384299,-0.344510,-0.185356,-0.380866,...,-0.382529,-0.308433,-0.056560,-0.166685,-0.443320,-0.155476,-0.363526,-0.387407,-0.360380,-0.000726,-0.348262,-0.389957,-0.387407,0.451627,-0.325767,-0.024445,0.034997,-0.334662,-0.350426,-0.389957,-0.362640,-0.407181,0.117163,-0.057459,-0.406117,-0.222857,-0.377893,-0.356286,-0.067737,0.031045,0.435440,-0.114485,-0.401184,0.259862,0.306437,-0.421342,-0.431936,0.231025,-0.046819,0.227184
31624,0.813570,-0.321171,1.000000,0.998417,0.402521,-0.046308,0.813570,0.866425,0.998799,-0.002396,0.766412,0.981692,0.871886,0.992397,0.964129,0.682769,0.635698,0.111435,0.821043,0.997793,0.663733,0.527196,0.271509,0.556031,-0.012648,-0.354971,0.564612,0.672213,0.448054,0.755309,0.998442,0.574562,0.802681,-0.040344,0.831767,0.284644,0.714450,0.616316,0.735454,0.928992,...,0.703127,0.969844,0.734246,0.922940,0.710320,0.672112,0.876853,0.891686,0.889083,0.851884,0.931872,0.855270,0.891686,0.211722,0.626859,0.427002,0.696137,0.810274,0.942845,0.820867,0.871978,0.871582,0.213867,0.705791,0.564693,0.922706,0.940312,0.912721,0.734982,0.604982,0.328213,0.543520,0.700295,0.288929,0.265743,0.777867,0.765361,0.529662,0.706904,0.521436
32376,0.809146,-0.299820,0.998417,1.000000,0.416405,-0.021063,0.809146,0.862350,0.997468,0.023561,0.756079,0.976831,0.868502,0.991431,0.957650,0.676059,0.624736,0.144605,0.814198,0.994747,0.655920,0.515593,0.290983,0.544657,-0.005191,-0.335273,0.554099,0.664848,0.430687,0.748005,0.996595,0.562793,0.794873,-0.050060,0.825443,0.304778,0.707516,0.603691,0.727907,0.925695,...,0.697870,0.974109,0.755914,0.935446,0.706351,0.662820,0.876453,0.888460,0.890047,0.848778,0.931393,0.850215,0.888460,0.252729,0.623652,0.422591,0.703226,0.811176,0.943596,0.815007,0.871359,0.869063,0.244929,0.704307,0.561089,0.932867,0.937579,0.911157,0.754094,0.627001,0.366787,0.549218,0.699350,0.281659,0.303587,0.775394,0.764239,0.554826,0.730903,0.514566
45370,0.603646,0.027397,0.402521,0.416405,1.000000,0.331710,0.603646,0.265705,0.382570,0.320756,0.554360,0.353348,0.596206,0.347759,0.334471,0.116115,0.432465,0.399455,0.606560,0.391587,0.482751,0.554673,0.053516,0.376067,0.012105,0.385154,0.351215,0.470648,0.401172,0.553163,0.382177,0.802135,0.206933,0.232481,0.282417,0.501216,0.64

In [29]:
product_sim_df

,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
short_term_deposit,1.000000,0.985296,0.987423,0.749588,0.739721,0.505670,-0.013758,-0.721439,-0.791690,-0.457159,-0.839971,0.993868,0.021773
loans,0.985296,1.000000,0.983642,0.743112,0.733349,0.494539,-0.004097,-0.717800,-0.787756,-0.458706,-0.834433,0.990058,0.018157
mortgage,0.987423,0.983642,1.000000,0.746314,0.735506,0.495303,-0.009075,-0.716622,-0.787493,-0.459742,-0.835803,0.992167,0.016371
funds,0.749588,0.743112,0.746314,1.000000,0.619301,0.547417,-0.061784,-0.687685,-0.730496,-0.308278,-0.758255,0.750767,0.145353
securities,0.739721,0.733349,0.735506,0.619301,1.000000,0.387306,-0.046277,-0.651614,-0.692457,-0.316340,-0.712341,0.741259,0.108068
long_term_deposit,0.505670,0.494539,0.495303,0.547417,0.387306,1.000000,-0.098234,-0.597354,-0.621458,-0.118280,-0.616712,0.500406,0.239637
credit_card,-0.013758,-0.004097,-0.009075,-0.061784,-0.046277,-0.098234,1.000000,-0.135873,-0.121492,-0.066111,-0.082540,-0.010316,-0.081653
pension_plan,-0.721439,-0.717800,-0.716622,-0.687685,-0.651614,-0.597354,-0.135873,1.000000,0.901949,0.004430,0.683533,-0.721448,-0.448980
payroll_account,-0.791690,-0.787756,-0.787493,-0.730496,-0.692457,-0.621458,-0.121492,0.901949,1.000000,0.092814,0.765772,-0.791910,-0.455206
emc_account,-0.457159,-0.458706,-0.459742,-0.308278,-0.316340,-0.118280,-0.066111,0.004430,0.092814,1.000000,0.183591,-0.459734,0.188062


Dado que hay muchos clientes similares por tener los mismos productos contratado, se opta por utilizar la matriz según la similitud de productos

##Recomendación product based

Miramos que tiene cada cliente contratado actualmente

In [30]:
target_clients_now = target_clients[target_clients['pk_partition']==target_clients['pk_partition'].max()]

In [31]:
target_clients_now

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster,total_mes,total_sum
118,1050686,2019-05-28,0,0,0,0,0,0,0,0.0,0,1,1,0,1,5,3.0,50.0
135,1050660,2019-05-28,0,0,0,0,0,0,0,1.0,1,1,1,0,0,5,5.0,76.0
169,1050651,2019-05-28,0,0,0,0,0,0,0,1.0,1,1,1,0,0,5,5.0,75.0
186,1050652,2019-05-28,0,0,0,0,1,0,0,0.0,1,1,1,0,0,5,4.0,78.0
203,1050659,2019-05-28,0,0,0,0,1,0,1,1.0,1,1,1,0,0,5,7.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866558,1433369,2019-05-28,0,0,0,0,0,0,1,1.0,1,1,1,0,0,4,6.0,56.0
867166,1428596,2019-05-28,0,0,0,0,0,0,1,1.0,1,1,1,0,0,4,6.0,51.0
869437,1437442,2019-05-28,0,0,0,0,1,1,0,1.0,1,0,1,0,1,4,7.0,59.0
883713,1442421,2019-05-28,0,0,0,0,0,0,1,1.0,1,0,1,0,1,4,6.0,50.0


Función para sacar los top productos que esta relacionado con otro producto

Esta función se utiliza más adelante y servirá para saber que 4 productos está más relacionado a un producto en concreto

In [32]:
def top_products(name, df):
    '''
    This functions prints top 10 similar products, based on the number of times that they were acquired.
    '''
    #print('Similar shows to {} include:\n'.format(name))
    
    index = product_sim_df[name].sort_values(ascending = False).index[1:11]
    values = product_sim_df[name].sort_values(ascending = False).values[1:11]

    return {index[0]:values[0],index[1]:values[1],index[2]:values[2],index[3]:values[3]}


Hacemos drop de las columnas que no necesitamos para la recomendación

In [33]:
targets_clients_now_df = target_clients_now.drop(['pk_partition','total_mes','total_sum','cluster'],axis=1)

In [34]:
target_clients_now[target_clients_now['total_mes']==0]

,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,credit_card,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster,total_mes,total_sum
20958,1045535,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,88.0
73030,1020461,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,68.0
74847,1020757,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,62.0
114193,1123356,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,55.0
128972,1122099,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,62.0
169503,1119050,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,75.0
176542,1116106,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,62.0
418412,1378440,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,55.0
565434,1139241,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,60.0
610001,1223301,2019-05-28,0,0,0,0,0,0,0,0.0,0,0,0,0,0,4,0.0,51.0


In [35]:
targets_clients_now_df = targets_clients_now_df.set_index('pk_cid')

Construimos un diccionario de que productos se recomendaría a cada cliente


*   Miramos que productos tiene contratado cada cliente a día de hoy (última fecha disponible)
*   Para cada uno de esos productos, nos sacamos los 4 productos que más se parece, que serán los que vamos a recomendar después



In [36]:
from collections import OrderedDict
def get_dic(df):
  dict_client = {}
  list_dict = []
  for cliente in df.index:
    list_client = []
    list_dict = []
    dict_product = {}
    for product,value in zip((df[df.index == cliente].any() == True).index,(df[df.index == cliente].any() == True).values) :
      if value == True:
        dict_top = top_products(product,product_sim_df)
        for element in dict_top:
          if element in dict_product:
            if dict_top[element] > dict_product[element]:
              dict_product[element] = dict_top[element]
          else:
            dict_product[element] = dict_top[element]
    sorted_d = OrderedDict( sorted(dict_product.items(),
                           key=lambda item: item[1],
                           reverse=True))
    dict_client[cliente] = sorted_d
  return dict_client

El diccionario de cada cliente y que productos le vamos a recomendar de mayor similitud a menor según los productos que tiene ese cliente contratado

In [37]:
diccionario_cliente = get_dic(targets_clients_now_df)

Construimos el dataframe con el cliente y que producto que vamos a recomendar

In [38]:
lista_recomendacion = pd.DataFrame(columns=['Cliente','Recomendacion'])
for cliente in targets_clients_now_df.index:
  for producto in diccionario_cliente[cliente]:
      if (targets_clients_now_df[targets_clients_now_df.index == cliente][producto].to_list())[0] == 0:
        lista_recomendacion = lista_recomendacion.append({'Cliente':cliente,'Recomendacion':producto},ignore_index = True)
        break
 

In [39]:
lista_recomendacion['Recomendacion'].value_counts()

emc_account           2993
pension_plan          1870
em_acount             1664
payroll_account       1298
debit_card             911
long_term_deposit      524
em_account_p           501
funds                  290
loans                    6
short_term_deposit       1
Name: Recomendacion, dtype: int64

Salen menos clientes porque hay clientes que no tienen ningún producto contratado y no se le puede asignar una recomendación 

In [40]:
lista_recomendacion

,Cliente,Recomendacion
0,1050686,payroll_account
1,1050660,em_acount
2,1050651,em_acount
3,1050652,pension_plan
4,1050659,em_account_p
...,...,...
10053,1433369,em_acount
10054,1428596,em_acount
10055,1437442,em_account_p
10056,1442421,long_term_deposit


Como hay clientes que no existen en el último pk_partition y hay otros que no tienen ningún producto contratado. Partiamos de 10100 clientes y el resultado es 10058. Haremos un drop de 58 clientes con em_acount_p que es el que nos da menos beneficio

In [41]:
len_lista=  len(lista_recomendacion)
for cliente in lista_recomendacion.Cliente:
  if len_lista != 10000:
    if (lista_recomendacion[lista_recomendacion.Cliente == cliente].Recomendacion == 'em_account_p').to_list()[0]:
      lista_recomendacion.drop(lista_recomendacion.loc[lista_recomendacion['Cliente']==cliente].index, inplace=True)
      len_lista = len_lista -1

In [42]:
lista_recomendacion['Recomendacion'].value_counts()

emc_account           2993
pension_plan          1870
em_acount             1664
payroll_account       1298
debit_card             911
long_term_deposit      524
em_account_p           443
funds                  290
loans                    6
short_term_deposit       1
Name: Recomendacion, dtype: int64